In [1]:
import torchvision
from torchvision import transforms
import torch.nn as nn
from torch.utils.data import DataLoader
import torch
import torch.nn.functional as F

In [2]:

device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [3]:
from matplotlib.pyplot import imshow
%matplotlib inline

In [24]:
trans = transforms.Compose([
    transforms.ToTensor()
])

train_data = torchvision.datasets.ImageFolder(root="train_data", transform=trans)

In [25]:
for n,value in enumerate(train_data):
    data, label = value
    print(n,data,label)
    if(label ==0):
        data.save('train_data/gray/%d_%d.jpeg'%(n, label))
    else:
        data.save('train_data/gray/%d_%d.jpeg'%(n, label))

0 tensor([[[0.7412, 0.7373, 0.7294,  ..., 0.5059, 0.5059, 0.5020],
         [0.7373, 0.7333, 0.7294,  ..., 0.4941, 0.4941, 0.4863],
         [0.7373, 0.7333, 0.7333,  ..., 0.4824, 0.4745, 0.4706],
         ...,
         [0.7725, 0.7569, 0.7412,  ..., 0.4431, 0.4471, 0.4510],
         [0.8039, 0.7804, 0.7529,  ..., 0.4392, 0.4353, 0.4392],
         [0.8314, 0.7961, 0.7608,  ..., 0.4314, 0.4275, 0.4275]],

        [[0.7412, 0.7373, 0.7294,  ..., 0.5255, 0.5255, 0.5216],
         [0.7373, 0.7333, 0.7294,  ..., 0.5137, 0.5137, 0.5059],
         [0.7373, 0.7333, 0.7333,  ..., 0.4902, 0.4824, 0.4784],
         ...,
         [0.9098, 0.8941, 0.8784,  ..., 0.4275, 0.4314, 0.4353],
         [0.9490, 0.9176, 0.8902,  ..., 0.4235, 0.4196, 0.4235],
         [0.9765, 0.9412, 0.8980,  ..., 0.4157, 0.4118, 0.4118]],

        [[0.7333, 0.7294, 0.7216,  ..., 0.5020, 0.5020, 0.4980],
         [0.7294, 0.7255, 0.7216,  ..., 0.4902, 0.4902, 0.4824],
         [0.7294, 0.7255, 0.7255,  ..., 0.4706, 0.4627, 

AttributeError: 'Tensor' object has no attribute 'save'

In [26]:
nn.Conv2d(3,6,kernel_size=5,stride=1)

Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))

In [27]:
train_data[0]

(tensor([[[0.7412, 0.7373, 0.7294,  ..., 0.5059, 0.5059, 0.5020],
          [0.7373, 0.7333, 0.7294,  ..., 0.4941, 0.4941, 0.4863],
          [0.7373, 0.7333, 0.7333,  ..., 0.4824, 0.4745, 0.4706],
          ...,
          [0.7725, 0.7569, 0.7412,  ..., 0.4431, 0.4471, 0.4510],
          [0.8039, 0.7804, 0.7529,  ..., 0.4392, 0.4353, 0.4392],
          [0.8314, 0.7961, 0.7608,  ..., 0.4314, 0.4275, 0.4275]],
 
         [[0.7412, 0.7373, 0.7294,  ..., 0.5255, 0.5255, 0.5216],
          [0.7373, 0.7333, 0.7294,  ..., 0.5137, 0.5137, 0.5059],
          [0.7373, 0.7333, 0.7333,  ..., 0.4902, 0.4824, 0.4784],
          ...,
          [0.9098, 0.8941, 0.8784,  ..., 0.4275, 0.4314, 0.4353],
          [0.9490, 0.9176, 0.8902,  ..., 0.4235, 0.4196, 0.4235],
          [0.9765, 0.9412, 0.8980,  ..., 0.4157, 0.4118, 0.4118]],
 
         [[0.7333, 0.7294, 0.7216,  ..., 0.5020, 0.5020, 0.4980],
          [0.7294, 0.7255, 0.7216,  ..., 0.4902, 0.4902, 0.4824],
          [0.7294, 0.7255, 0.7255,  ...,

In [28]:
data_loader = DataLoader(dataset=train_data, batch_size=8, shuffle=True, num_workers=2)

In [73]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3,6,kernel_size=5, stride=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(6,16,kernel_size=5, stride=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.layer3 = nn.Sequential(
            nn.Linear(16*13*29,120),
            nn.ReLU(),
            nn.Linear(120,2)
        )
        
        
    def forward(self, x):
        out = self.layer1(x)
#         print(out.shape)
        out = self.layer2(out)
#         print(out.shape)
        out = out.view(out.shape[0],-1)
#         print(out.shape)
        out = self.layer3(out)
        return out

In [74]:
net = CNN().to(device)
test_input = (torch.Tensor(3,3,64,128)).to(device)
test_out = net(test_input)

In [75]:
optimizer = torch.optim.Adam(net.parameters(), lr=0.0005)
criterion = nn.CrossEntropyLoss().to(device)

In [76]:
#training
total_batch = len(data_loader)

epochs = 3
for epoch in range(epochs):
    avg_cost=0
    
    for num, data in enumerate(data_loader):
        imgs, labels = data
        imgs = imgs.to(device)
        labels = labels.to(device)

        
        prediction = net(imgs)
        cost = criterion(prediction,labels)
        
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()
        
        avg_cost += cost/total_batch
    print(f'Epoch {epoch}, Cost = {avg_cost}')
    
print('learning finish')

Epoch 0, Cost = 0.02129351533949375
Epoch 1, Cost = 0.0
Epoch 2, Cost = 0.0
learning finish


In [77]:
torch.save(net.state_dict(), './train_data/model.pth')

In [78]:
new_net = CNN().to(device)

In [79]:
#불필요한 학습 제거
new_net.load_state_dict(torch.load('./train_data/model.pth'))

<All keys matched successfully>

In [80]:
print(net.layer1[0])
print(new_net.layer1[0])

print(net.layer1[0].weight[0][0][0])
print(new_net.layer1[0].weight[0][0][0])

Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
tensor([ 0.0053, -0.1015, -0.0810,  0.1052,  0.0289], grad_fn=<SelectBackward>)
tensor([ 0.0053, -0.1015, -0.0810,  0.1052,  0.0289], grad_fn=<SelectBackward>)


In [81]:
trans = transforms.Compose([
    transforms.Resize((64,128)),
    transforms.ToTensor()
])

test_data = torchvision.datasets.ImageFolder(root="test_data", transform=trans)

In [82]:
test_set = DataLoader(dataset=test_data, batch_size=len(test_data))

In [84]:
with torch.no_grad():
    for num, data in enumerate(test_set):
        imgs, labels = data
        imgs = imgs.to(device)
        labels = labels.to(device)
        
        prediction = net(imgs)
        
        correct_prediction = torch.argmax(prediction,1) == labels
        
        acc = correct_prediction.float().mean()
        print('acc: ',acc.item())

acc:  0.41999998688697815
